In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

#requesting item information from the API
api_url = "https://api.warframe.market/v1/items"
headers = {"Authorization": "Bearer YOUR_ACCESS_TOKEN"}
response = requests.get(api_url, headers=headers)

#check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()['payload']['items']
    df = pd.DataFrame(data)

else:
    #error responses
    print(f"Error: {response.status_code} - {response.text}")

#initializing columns for lowest price and predicted lowest price for the next day
df['lowest price available'] = 0
df['predicted lowest price for next day'] = 0

#keyword to filter items
keywords = ['prime']

#loop through each item in the DataFrame
for i in range(len(df['url_name'])):
    item_name = df['url_name'][i]
    
    #check if item_name contain the keyword
    if all(keyword in item_name for keyword in keywords):
        
        #requesting specific item information from the API
        api_url = f"https://api.warframe.market/v1/items/{item_name}/orders"
        headers = {"Authorization": "Bearer teehee"}
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            data = response.json()['payload']['orders']
            df1 = pd.DataFrame(data)

            #initialize lists to store user information
            user_ids = []
            statuses = []

            for order in data:
                user_data = order['user']
                user_ids.append(user_data['id'])
                statuses.append(user_data['status'])

            df1['user_id'] = user_ids
            df1['status'] = statuses

        else:
            print(f"Error: {response.status_code} - {response.text}")

        #filter orders for in game status
        df1 = df1.loc[(df1['status'] == "ingame")]
        
        #extracting order information
        df1['t1'] = pd.to_datetime(df1['last_update'])
        df1 = df1.loc[(df1['t1'].dt.year == 2024) & (df1['order_type'] == "sell")]
        mini = df1['platinum'].min()
        df1 = df1.sort_values(by='platinum', ascending=True)

        #filter orders within a certain platinum range
        df1 = df1.loc[(df1['platinum'] < (mini + 20))]
        df['lowest price available'][i] = mini

        #only predicting items with more than 5 orders
        if df1['id'].count() > 5:
            df1['t1'] = pd.to_datetime(df1['last_update'])
            df1['date_numeric'] = df1['t1'].view('int64') // 10**9

            df1['day'] = df1['t1'].dt.day
            day = ['day']
            x = df1[day]
            y = df1['platinum']

            #training and testing sets
            xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=20)
            
            #initialize and train the linear regression model
            model = LinearRegression()
            model.fit(xtrain, ytrain)

            #making predictions based on the test set
            ypred = model.predict(xtest)

            #predict for the next day
            z=dt.datetime.today().strftime("%d")
            next_day = pd.DataFrame({'day': [int(z)+1]})  
            next_day_prediction = model.predict(next_day)
            
            df['predicted lowest price for next day'][i] = round(next_day_prediction[0])

        #for no orders are available, we set it as 'NA'
        elif df1['id'].count() == 0:
            df['predicted lowest price for next day'][i] = 'NA'

        #for less than 5 orders, we take the minimum
        else:
            df['predicted lowest price for next day'][i] = df1['platinum'].min()

        del df1
            
    #drop rows that do not meet the keyword criteria
    else:
        df.drop(i, inplace=True)

#save the dataframe into an excel file
excel_file_path = "warframe_market_data2.xlsx"
df.to_excel(excel_file_path, index=False)
